First of all we are going to download the data

In [1]:
import pandas as pd

data = pd.read_csv(r'C:\Users\crist\mentoring\comptes des communes\data\raw\base_communes.csv', sep=";", encoding="utf-8")

C:\Users\crist\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4,14,18) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


The dimensions of our dataframe:

In [2]:
data.shape

(18120499, 42)

We are going to start an analysis without any previous knowlegde of the subject. 
Thus, the first thing we will do will be to look for the general information of our database. 
If necesary, we will search the fields in the internet, trying to have some extra help.

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18120499 entries, 0 to 18120498
Data columns (total 42 columns):
 #   Column                                 Dtype  
---  ------                                 -----  
 0   exer                                   int64  
 1   outre_mer                              object 
 2   reg_code                               float64
 3   reg_name                               object 
 4   dep_code                               object 
 5   dep_name                               object 
 6   epci_code                              float64
 7   epci_name                              object 
 8   tranche_population                     float64
 9   rural                                  object 
 10  montagne                               object 
 11  touristique                            object 
 12  tranche_revenu_imposable_par_habitant  float64
 13  qpv                                    object 
 14  com_code                               object 
 

In [34]:
data.describe()

,exer,tranche_population,tranche_revenu_imposable_par_habitant,siren,ident,montant,ptot,euros_par_habitant,presence_budget,cbudg,agregat_niveau
count,1.812050e+07,1.810849e+07,1.810849e+07,1.812050e+07,1.812050e+07,1.812050e+07,1.810817e+07,1.810660e+07,18108491.0,1.812050e+07,1.812050e+07
mean,2.014929e+03,2.922191e+00,1.328479e+00,2.144496e+08,2.144496e+13,4.051122e+05,3.568466e+03,1.815727e+02,1.0,1.860787e+00,2.588223e+00
std,2.213752e+00,1.783196e+00,7.049265e-01,2.993979e+06,2.993979e+11,1.132366e+07,4.357441e+04,5.365039e+02,0.0,9.902625e-01,1.675742e+00
min,2.012000e+03,0.000000e+00,0.000000e+00,2.000022e+08,2.000022e+13,-9.435051e+08,0.000000e+00,-4.169737e+04,1.0,1.000000e+00,0.000000e+00
25%,2.013000e+03,2.000000e+00,1.000000e+00,2.124034e+08,2.124034e+13,0.000000e+00,2.570000e+02,0.000000e+00,1.0,1.000000e+00,1.000000e+00
50%,2.015000e+03,3.000000e+00,1.000000e+00,2.146015e+08,2.146015e+13,1.528010e+04,6.180000e+02,2.854750e+01,1.0,1.000000e+00,3.000000e+00
75%,2.017000e+03,3.000000e+00,2.000000e+00,2.167032e+08,2.167032e+13,1.045579e+05,1.636000e+03,1.782688e+02,1.0,3.000000e+00,4.000000e+00
max,2.020000e+03,1.000000e+01,5.000000e+00,2.197402e+08,2.197402e+13,6.845968e+09,2.274880e+06,1.827898e+05,1.0,3.000000e+00,6.000000e+00


### DEFINITIONS

- epci: Établisements publics de coopération intercommunale
- qpv: Quartier de la politique de la ville
- Tranche population: répartition de la population par age
- tranche_revenu_imposable_par_habitant: tranche correspondant à des taux d'imposition (Il y en a 5)
- Siren: système informatique pour le répertoire des entreprises et des établissements (unique identifier)
- insee: Le code INSEE (Institute for Statistics)
- ident ??? quelque sort d'identifiant mais pas connu. Il y en a 84952 différents
- lbudg: localitation budget??
- nomen: Nomenclatures comptables (números de cuentas)
- agregat: grandeur synthétique mesurant le résultat de l'activité économique.Le principal agrégat est le PIB. Macrodatos
- ptot: Poblation total
- presence_budget: Il n'y a que la valeur 1. À éliminer?
- cbudg ?? Il n'y a que les valeurs 1 et 3
- agregat: Agregat description
- agregat niveau: Il y en a 7 niveau
- ordre d'affichage ¿? à éliminer?

The first thing we should do is to clean data. Without knowing anything about the subject, it is always better to mantain all the fields, but in this case we can see clearly that there are some entries that are duplicated:
- reg_code/reg_name
- dep_code/ dep_name
- Epci_code/ epci_name
- Com_code/ com_name
- Montant/ montant_en_millions

We are going to eliminate all the codes, keeping only the names.

In [2]:
data = data.drop(['reg_code', 'dep_code', 'epci_code','com_code', 'montant_en_millions'], axis=1)

Do we have missing values? As we are working with a big dataframe it is important to know if we have so many rows with missing values for all the variables. In this case, we could eliminate lines easing our job.  

In [2]:
data.isnull().sum(axis = 0)

exer                                            0
outre_mer                                   12008
reg_code                                    12008
reg_name                                    12326
dep_code                                    12008
dep_name                                    12326
epci_code                                   18145
epci_name                                   18463
tranche_population                          12008
rural                                       12008
montagne                                    12008
touristique                                 12008
tranche_revenu_imposable_par_habitant       12008
qpv                                         12008
com_code                                    12008
com_name                                    12008
categ                                           0
siren                                           0
insee                                       12008
ident                                           0


In a first study we are going to eliminate as well all the columns ordre_analyseX_sectionX, cause they are descriptibe variables that do not give us so many information and because they conteins a huge amount of missings
.   

In [3]:
#data['ordre_analyse1_section1','ordre_analyse1_section2','ordre_analyse1_section3','ordre_analyse2_section1','ordre_analyse2_section2',
#'ordre_analyse2_section3','ordre_analyse3_section1','ordre_analyse3_section2','ordre_analyse3_section3','ordre_analyse4_section1',
#'ordre_affichage'].isnull().sum(axis = 0)

data =data.drop(['ordre_analyse1_section1','ordre_analyse1_section2','ordre_analyse1_section3','ordre_analyse2_section1','ordre_analyse2_section2',
'ordre_analyse2_section3','ordre_analyse3_section1','ordre_analyse3_section2','ordre_analyse3_section3','ordre_analyse4_section1',
'ordre_affichage'], axis = 1)

In [5]:
data.shape

(18120499, 26)

We have reduced the variables from 42 to 26, reducing a little bit the processing time and consequently our job.

After this study we can note that we have 13 variables with 12008 missing values and this missing values in some way are related, because with the com_name we can find the dep_name, the epci_name, the reg_name and if it is outre_mer.
Hence, we are going to create a dataframe filtering by outre_mer missing data


In [7]:
outre_mer_missings = data['outre_mer'].isnull()
outre_mer_missings = pd.DataFrame(outre_mer_missings)
om = outre_mer_missings[outre_mer_missings['outre_mer']==True]
indice = om.index
outre_mer_missings = data.iloc[indice]
outre_mer_missings

,exer,outre_mer,reg_name,dep_name,epci_name,tranche_population,rural,montagne,touristique,tranche_revenu_imposable_par_habitant,...,lbudg,type_de_budget,nomen,agregat,montant,ptot,euros_par_habitant,presence_budget,cbudg,agregat_niveau
2544,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,BOURG-DE-THIZY,Budget principal,M14,Dépenses d'intervention,3.876198e+05,NaN,NaN,NaN,1,4
2545,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,POMPES FUNEBRES BOURG-DE-THIZY,Budget annexe,M4,Dépenses d'intervention,0.000000e+00,NaN,NaN,NaN,3,4
2575,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,CHAPELLE-DE-MARDORE (LA ),Budget principal,M14,Dépenses d'intervention,8.744170e+03,NaN,NaN,NaN,1,4
2576,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,COMMERCE CAFE LA-CHAPELLE-DE-MARDORE,Budget annexe,M4,Dépenses d'intervention,0.000000e+00,NaN,NaN,NaN,3,4
2732,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,MARDORE,Budget principal,M14,Dépenses d'intervention,2.124477e+04,NaN,NaN,NaN,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17866218,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,GOUSSAINVILLE,Budget principal,M14,Autres recettes de fonctionnement,2.325119e+04,NaN,NaN,NaN,1,4
17866219,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,ASST-GOUSSAINVILLE,Budget annexe,M49,Autres recettes de fonctionnement,-1.455192e-11,NaN,NaN,NaN,3,4
17874703,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,BADINIERES,Budget principal,M14,Autres recettes de fonctionnement,1.649631e+04,NaN,NaN,NaN,1,4
17874704,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,LOT LE LOUP BADINIERES,Budget annexe,M14,Autres recettes de fonctionnement,0.000000e+00,NaN,NaN,NaN,3,4


We can draw some important information from this dataframe. 
Firstly, we confirm that from the field lbudg we can obtain the com_name. The patterns that we have found are the followings:
- If there are only one word, this is directly the name that we were searching for, so just enough to copy from lbudg to com_name.
- When the com_name is composed it goes always with a dash
- When there are more than a word and the com_name is not composed the com_name is the last word.

In [8]:
import re
#r3 = re.compile(r"[^-]+$") from - ahead

# Composed names
r1 = re.compile(r"([a-zA-Z0-9.-áè]+-[a-zA-Z'.-]+)")

# One unique name or several names (we keep the last one)

r2 = re.compile(r"[^\W]+$")

for i, row in outre_mer_missings.iterrows():
    outre_mer_missings.at[i,'com_name'] = r1.findall(outre_mer_missings.at[i,'lbudg'])
for i, row in outre_mer_missings.iterrows():
    if not row['com_name']:
        outre_mer_missings.at[i,'com_name'] = r2.findall(outre_mer_missings.at[i,'lbudg'])

for i, row in outre_mer_missings.iterrows():
    if not row['com_name']:
         outre_mer_missings.at[i,'com_name'] = row['lbudg']
    
# We undo the list that is automatically done.
       
for i, row in outre_mer_missings.iterrows():
    outre_mer_missings.at[i,'com_name'] = "".join(row['com_name']) 

# We have detected that in some cases there are some words "RESTAURANT", "EAU", "ASSAT" following for a dash that are in our row and we did not want it.
#So, we will remove it.
words = ["ASST-", "RESTAURANT-", "EAU-", "LOTS-", "FORETS-", "1-", "HBS-", "SPANC-", "LOT-"]
for i, row in outre_mer_missings.iterrows():
    for word in words:
        if word in row['com_name']:
            outre_mer_missings.at[i,'com_name'] = row['com_name'].replace(word, "")
            
# Now, as in our dataframe "data" we have the column com_name in lowercase, we are going to transform it:
# For that the easiest way is to convert into lowercase and then make a title:
for i,row in outre_mer_missings.iterrows():
    outre_mer_missings.at[i,'com_name'] = row['com_name'].lower()
    outre_mer_missings.at[i,'com_name'] = row['com_name'].title()
    
# we have identified at the end of some words the articles that is necesary to eliminate ['(Le )', '(La )', '(Les)']

for i, row in outre_mer_missings.iterrows():
    if '(Le )' in row['com_name']:
        row['com_name'] = row['com_name'].replace(' (Le )', "")
        outre_mer_missings.at[i,'com_name'] = "Le" + " " +  row['com_name']
    if '(La )' in row['com_name']:
        row['com_name'] = row['com_name'].replace(' (La )', "")
        outre_mer_missings.at[i,'com_name'] = "La" + " " +  row['com_name']
    if '(Les)' in row['com_name']:
        row['com_name'] = row['com_name'].replace(' (Les)', "")
        outre_mer_missings.at[i,'com_name'] = "Les" + " " +  row['com_name']

# At this point we can look for the unique names to compare and change manually some names if it is necessary

for i, row in outre_mer_missings.iterrows():
    if "Magny" in row['com_name']:
        outre_mer_missings.at[i,'com_name'] = "Le Magny"
    if 'Chapelle-De-Mardore' in row['com_name']:
        outre_mer_missings.at[i,'com_name'] = 'La-Chapelle-De-Mardore'
    if "Bihorel" in row['com_name']:
         outre_mer_missings.at[i,'com_name'] = "Bois-Guillaume-Bihorel"
    if "Dessous" in row['com_name']:
        outre_mer_missings.at[i,'com_name'] = "Saint-Offenge-Dessous"
    if "Chedoue" in row['com_name']:
        outre_mer_missings.at[i,'com_name'] = "Fresnaye-Sur-Chedouet"


To move fordward with our analysis, ones we have found all the com_name missings, we are going to search the com_name that we have in our dataframe "data". For those com_name from "outre_mer_missings" that are in "data" we can extract the fields reg_name, dep_name, epci_name and outre_mer. 

In [11]:
com_name_om = outre_mer_missings['com_name'].unique()

com_name_data = data['com_name'].unique()

common = []
for element in com_name_om:
    if element in com_name_data:
        common.append(element)

common

['Thizy',
 'Vaugneray',
 'La Villeneuve',
 'Montigny',
 'Bois',
 'Le Magny',
 'Saint-Bonnet',
 'Corbon',
 'Champagne',
 'Goussainville',
 'Melay',
 'Larchamp',
 'Vrigny']

We are going to study deeply one of this communes chosen randomly:

In [13]:
Goussainville=data.loc[data['com_name']=='Goussainville']
Goussainville

,exer,outre_mer,reg_name,dep_name,epci_name,tranche_population,rural,montagne,touristique,tranche_revenu_imposable_par_habitant,...,lbudg,type_de_budget,nomen,agregat,montant,ptot,euros_par_habitant,presence_budget,cbudg,agregat_niveau
19477,2012,Non,Île-de-France,Val-d'Oise,CA Roissy Pays de France,8.0,Non,Non,Non,1.0,...,GOUSSAINVILLE,Budget principal,M14,Dépenses d'intervention,2063634.25,31225.0,66.089167,1.0,1,4
19478,2012,Non,Île-de-France,Val-d'Oise,CA Roissy Pays de France,8.0,Non,Non,Non,1.0,...,SERV EAU GOUSSAINVILLE,Budget annexe,M49,Dépenses d'intervention,0.00,31225.0,0.000000,1.0,3,4
19479,2012,Non,Île-de-France,Val-d'Oise,CA Roissy Pays de France,8.0,Non,Non,Non,1.0,...,SERV ASST GOUSSAINVILLE,Budget annexe,M49,Dépenses d'intervention,0.00,31225.0,0.000000,1.0,3,4
93546,2012,Non,Île-de-France,Val-d'Oise,CA Roissy Pays de France,8.0,Non,Non,Non,1.0,...,GOUSSAINVILLE,Budget principal,M14,Dépenses d'investissement hors remb,13992751.45,31225.0,448.126548,1.0,1,3
93547,2012,Non,Île-de-France,Val-d'Oise,CA Roissy Pays de France,8.0,Non,Non,Non,1.0,...,SERV EAU GOUSSAINVILLE,Budget annexe,M49,Dépenses d'investissement hors remb,241583.05,31225.0,7.736847,1.0,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18019879,2015,Non,Centre-Val de Loire,Eure-et-Loir,CC du Pays Houdanais (C.C.P.H.),3.0,Oui,Non,Non,2.0,...,GOUSSAINVILLE,Budget principal,M14,Encours de dette - Dettes bancaires et assimilées,157250.44,1275.0,123.333678,1.0,1,2
18054786,2015,Non,Île-de-France,Val-d'Oise,CA Roissy Pays de France,8.0,Non,Non,Non,1.0,...,GOUSSAINVILLE,Budget principal,M14,Encours de dette - Dettes bancaires et assimilées,51351675.99,31525.0,1628.919143,1.0,1,2
18054787,2015,Non,Île-de-France,Val-d'Oise,CA Roissy Pays de France,8.0,Non,Non,Non,1.0,...,EAU - GOUSSAINVILLE,Budget annexe,M49,Encours de dette - Dettes bancaires et assimilées,235531.04,31525.0,7.471246,1.0,3,2
18075528,2015,Non,Centre-Val de Loire,Eure-et-Loir,CC du Pays Houdanais (C.C.P.H.),3.0,Oui,Non,Non,2.0,...,GOUSSAINVILLE,Budget principal,M14,Epargne de gestion,131087.36,1275.0,102.813616,1.0,1,0


We can draw from that many important conclusions.

- For the same commun, we can have different reg_name, dep_name, epci_name
- This differences depends on the siren

We need therefore the different siren and we will search in our dataframe data to extract the missings values.

In [15]:
data_siren = data['siren'].unique()
om_siren = outre_mer_missings['siren'].unique()
common_siren = []
for element in om_siren:
    if element in data_siren:
        common_siren.append(element)
        
# We find the position of each siren in the dataframe data, to search then for the rest of the fields that we need.
# For that it is necesary to build a dataframe without the values in the dataframe outre_mer_missings
local = []
for element in common_siren:
    loc = data[data['siren']==element].index[0]
    local.append(loc)
    
# The problem is that we have not the same siren in the dataframe data, so, we could not find by this method the missings fields.
#We are going to try with the comm_name

37528

In [17]:
com_name_data = data['com_name'].unique()
com_name_om = outre_mer_missings['com_name'].unique()
common = []
for element in com_name_om:
    if element in com_name_data:
        common.append(element)
        

outre_mer_ind = outre_mer_missings.index

sol=[]
for element in common:   
    possible_ind = list(data[data['com_name']==element].index)
    for e in possible_ind:
        if e not in outre_mer_ind:
            loc = data.loc[e]['outre_mer']
    sol.append(loc)
sol

['Non',
 'Non',
 'Non',
 'Non',
 'Non',
 'Non',
 'Non',
 'Non',
 'Non',
 'Non',
 'Non',
 'Non',
 'Non']

For the 13 commune names that we had we can see that the field outre_mer is non, we can therefore fill it.
    

In [18]:
for i, row in outre_mer_missings.iterrows():
    outre_mer_missings.at[i, 'outre_mer'] == "Non"

In [19]:
outre_mer_missings.outre_mer


2544        NaN
2545        NaN
2575        NaN
2576        NaN
2732        NaN
           ... 
17866218    NaN
17866219    NaN
17874703    NaN
17874704    NaN
17874922    NaN
Name: outre_mer, Length: 12008, dtype: object